In [ ]:
##กรมบังคับคดี #ค้นหาแปลงที่ดิน
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import csv
import sys

options = webdriver.ChromeOptions() 
options.add_argument("headless")
driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)   
    
def find_percen_province(province):
    url = 'http://asset.led.go.th/newbidreg/'
    driver.get(url)
 
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div/div[3]/div/div[1]/table/tbody/tr/td[2]/select').send_keys(province)

    prov_percen = driver.current_url.split('=')[-1]

    return prov_percen

def find_maxpage(province):
    prov_percen = find_percen_province(province)
    
    page = 1
    url = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)

    driver.get(url)
    max_page = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[2]/table[1]/tbody/tr/td[2]/div').text.split('/')[-1]
    print('max_page : ',max_page)
    return int(max_page)

    
def read_table_data(province,page):
    prov_percen = find_percen_province(province)
    dict_data = []

    print('page : ',page)
    url = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)

    driver.get(url)

    html = driver.page_source

    sup = BeautifulSoup(html, "html.parser").find("table",{"class":"table linkevent"}).find_all('tr')[2:]

    for i in sup:
        link = 'http://asset.led.go.th/newbidreg/'
        link += i['onclick'][13:-130].replace("ผบ", "%BC%BA").replace('บางส่วน','%BA%D2%A7%CA%E8%C7%B9').replace(' ','%20').replace('ไม่ปรากฏเลขทะเบียน','%E4%C1%E8%BB%C3%D2%A1%AF%E0%C5%A2%B7%D0%E0%BA%D5%C2%B9').replace('ม','C1')

        Law_Court_ID = link.split('&')[-3].split('=')[-1]
        deed_no = link.split('&')[-2].split('=')[-1].replace('%20','')
        addrno = link.split('&')[-1].split('=')[-1]

        ii = i.find_all('td')
        data = []
        for x in ii:
            data.append(x.text.strip())
               
        aumper = data[9]
        choose_one_deed_no = deed_no.replace('(',',').replace(')',',').replace('-',',').replace('.',',').split(',')[0]
        
        try:
            lat,long,xprice_gov,xarea1,xarea2,xarea3 = find_gps(province,aumper,choose_one_deed_no)
        except:
            print('-'*15)
            print("Unexpected error:", sys.exc_info()[0])
            print('deed_no :',deed_no)
            print('page :',page,'size3 :',data[6],'-----',province,aumper,choose_one_deed_no)
            print('-'*15)
            
            lat,long,xprice_gov,xarea1,xarea2,xarea3 = ['','','','','','']
            try:
                time.sleep(1)
                driver.switch_to_alert().accept();
            except:
                print('-'*10)
                print('error in aleart')
                print('-'*10)
                pass
                       
            url = 'http://dolwms.dol.go.th/tvwebp/'
            driver.get(url)

        prop_data = {
            'Law_Court_ID' : Law_Court_ID,
            'deed_no' : deed_no,
            'addrno' : addrno,
            'year' : data[0],
            'order_sell' : data[1],
            'case_id' : data[2],
            'prop_type' : data[3],
            'size1' : data[4],
            'size2' : data[5],
            'size3' : data[6],
            'price' : data[7],
            'tumbon' : data[8],
            'aumper' : aumper,
            'link' : link,
            'lat' : lat,
            'long' : long,
            'xprice_gov' : xprice_gov,
            'xarea1' : xarea1,
            'xarea2' : xarea2,
            'xarea3' : xarea3
        }

        print('deed_no :',prop_data['deed_no'],'price :',prop_data['price'],'lat :',prop_data['lat'],'long :',prop_data['long'])
        dict_data.append(prop_data)

    return dict_data
    
def write_head_csv(file_name,csv_columns):
    csv_file = file_name
    with open(csv_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()

def write_csv(file_name,page_data,csv_columns):
    try:
        with open(file_name, 'a') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
#             writer.writeheader()
            for data in page_data:
                writer.writerow(data)
            print('write to .csv')
            
    except IOError:
        print("I/O error")
        
        
def find_gps(province,aumpur_,deed_no):
    aumpur_ = aumpur_.split('(')[0]
    a = ['01 - เมืองนนทบุรี','02 - บางกรวย','03 - บางใหญ่','04 - บางบัวทอง','05 - ไทรน้อย','06 - ปากเกร็ด','บ1 - บางปลา','ป1 - ตลาดขวัญ','พ1 - ตลิ่งชัน','พ7 - ตลาดขวัญ','พ8 - บางซื่อ']
    aumpur = aumpur_
    for i in a:
        if aumpur_ == i.split('-')[-1].strip():
            aumpur = i  
    if aumpur == aumpur_:
        print('-'*10)
        print('unmatch aumper :',aumpur)
        print('-'*10)
  
    url = 'http://dolwms.dol.go.th/tvwebp/'

    driver.get(url)
    driver.find_element_by_xpath('//*[@id="form1"]/div[6]/div/a[2]').click()
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="ddlProvince"]').send_keys(province)
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="ddlAmphur"]').send_keys(aumpur)
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="txtPacelNo"]').send_keys(deed_no)
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="btnFind"]').click()

    time.sleep(2)
    d = driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div').text
    d = d.split('\n')

    map_data = {
        'lat' : d[-6].split(',')[0].split(':')[1].strip(),
        'long' : d[-6].split(',')[1],
        'price_gov' : d[-8].split()[2],
        'area1' : d[-12].split()[2],
        'area2' : d[-12].split()[4],
        'area3' : d[-12].split()[6],
    }
       
    return map_data['lat'],map_data['long'],map_data['price_gov'],map_data['area1'],map_data['area2'],map_data['area3']

        
#main    
province = 'นนทบุรี'
file_name = "land_case2.csv"
csv_columns = ['Law_Court_ID','deed_no','addrno','year','order_sell','case_id','prop_type','size1','size2','size3','price','tumbon','aumper','link','lat', 'long', 'xprice_gov', 'xarea1', 'xarea2', 'xarea3']
                                                                                                                                                               
write_head_csv(file_name,csv_columns)
max_page = find_maxpage(province)

for page_number in range(1,max_page):
    page_data = read_table_data(province,page_number)
    
    print(page_data)
    
    write_csv(file_name,page_data,csv_columns)

    

<ipython-input-1-588c6f93cc37>:14: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)


max_page :  35
page :  1
deed_no : 160385,160386,160387,160388,160389,160390,160393,160394,160395,160396,160397,160 price : 367,840.00 lat : 13.9237574536943 long : 100.4168994027
deed_no : 41299 price : 931,000.00 lat : 13.8914598689291 long : 100.4361107209
deed_no : 228106 price : 1,886,280.00 lat : 13.8617718192007 long : 100.4666147135
---------------
Unexpected error: <class 'selenium.common.exceptions.UnexpectedAlertPresentException'>
deed_no : 415104151141514
page : 1 size3 : 31.91 ----- นนทบุรี ปากเกร็ด 415104151141514
---------------


<ipython-input-1-588c6f93cc37>:79: DeprecationWarning: use driver.switch_to.alert instead
  driver.switch_to_alert().accept();


----------
error in aleart
----------
deed_no : 415104151141514 price : 303,145.00 lat :  long : 
deed_no : 68557 price : 1,023,500.00 lat : 13.9361549675151 long : 100.5123114300
deed_no : 230868 price : 4,244,500.00 lat : 13.8706559317939 long : 100.5413897284
deed_no : 8801,139483 price : 1,195,560.00 lat : 13.8686911738563 long : 100.4683950619
deed_no : 66448 price : 1,620,000.00 lat : 13.9081902993764 long : 100.3692995762
---------------
Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>
deed_no : 66449
page : 1 size3 : 64 ----- นนทบุรี บางใหญ่ 66449
---------------
----------
error in aleart
----------
deed_no : 66449 price : 1,280,000.00 lat :  long : 
deed_no : 66457 price : 1,220,000.00 lat : 13.9082786488564 long : 100.3708596703
deed_no : 66480 price : 58,420,000.00 lat : 13.9076442286956 long : 100.3708909654
deed_no : 66497 price : 58,420,000.00 lat : 13.907465327204 long : 100.3708996401
deed_no : 66498 price : 58,420,000.00 lat : 13.907637602

page :  2
deed_no : 66558 price : 58,420,000.00 lat : 13.9067852172784 long : 100.3700532631
deed_no : 66542 price : 58,420,000.00 lat : 13.9068421225363 long : 100.3707340623
deed_no : 66502 price : 58,420,000.00 lat : 13.9074800429203 long : 100.3712565443
deed_no : 66516 price : 1,640,000.00 lat : 13.9072565782118 long : 100.3693553875
deed_no : 66422 price : 1,659,040.00 lat : 13.9089966225056 long : 100.3708219085
deed_no : 181225,181226,181227,181228,181229 price : 196,000.00 lat : 13.8806968203405 long : 100.5287293178
deed_no : 6636 price : 4,221,000.00 lat : 13.9442259547904 long : 100.4788715974
deed_no : 82547 price : 3,750,000.00 lat : 13.9145026577229 long : 100.4334837664
deed_no : 2768 price : 1,796,290.00 lat : 13.8418673708246 long : 100.5112989365
deed_no : 79408 price : 1,862,400.00 lat : 13.8338584348759 long : 100.5043287822
deed_no : 37797 price : 4,451,572.00 lat : 13.9283645451963 long : 100.4631773378
deed_no : 149458 price : 1,237,172.40 lat : 13.9563120827233

page :  3
deed_no : 151795 price : 840,420.00 lat : 13.9736276187513 long : 100.4134274230
deed_no : 166338 price : 2,027,850.00 lat : 13.9782145438535 long : 100.4130865340
deed_no : 27649 price : 2,812,920.00 lat : 13.9526348056158 long : 100.4628043396
deed_no : 81943 price : 1,093,500.00 lat : 13.9600446698769 long : 100.4021310872
---------------
Unexpected error: <class 'selenium.common.exceptions.UnexpectedAlertPresentException'>
deed_no : 183964
page : 3 size3 : 58.9 ----- นนทบุรี บางบัวทอง 183964
---------------
----------
error in aleart
----------
deed_no : 183964 price : 931,990.00 lat :  long : 
deed_no : 156245 price : 1,129,114.80 lat : 13.9087772825951 long : 100.3983014663
deed_no : 34586 price : 1,184,942.50 lat : 13.9403426460063 long : 100.5457363877
deed_no : 6589 price : 180,125.00 lat : 13.8356374130816 long : 100.4428412344
deed_no : 150763 price : 8,151,250.00 lat : 13.8875707780291 long : 100.4367993345
deed_no : 140303 price : 4,186,000.00 lat : 13.8802191560

In [7]:
import csv
# csv_columns = ['Law_Court_ID','deed_no','addrno','year','order_sell','case_id','prop_type','size1','size2','size3','price','tumbon','aumper','link','lat', 'long', 'xprice_gov', 'xarea1', 'xarea2', 'xarea3']
csv_columns = ['Law_Court_ID','deed_no','addrno','year','order_sell','case_id','prop_type','size1','size2','size3','price','tumbon','aumper','link','lat', 'long', 'xprice_gov', 'xarea1', 'xarea2', 'xarea3']


page_data = [{'Law_Court_ID': '121', 'deed_no': '160385,160386,160387,160388,160389,160390,160393,160394,160395,160396,160397,160', 'addrno': '351/56', 'year': '4/2564 - 4/2564', 'order_sell': '7 - 1', 'case_id': 'ผบ.14800/2561', 'prop_type': 'ห้องชุด', 'size1': '0', 'size2': '0', 'size3': '33.44', 'price': '367,840.00', 'tumbon': 'บางบัวทอง', 'aumper': 'บางบัวทอง', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.14800&law_suit_year=2561&Law_Court_ID=121&deed_no=160385,160386,160387,160388,160389,160390,160393,160394,160395,160396,160397,160&addrno=351/56', 'lat': '13.9237574536943', 'long': '100.4168994027', 'xprice_gov': 'กรุณาตรวจสอบข้อมูลผ่านทาง', 'xarea1': '0', 'xarea2': '0', 'xarea3': '10.0'}, {'Law_Court_ID': '101', 'deed_no': '41299', 'addrno': '189/219', 'year': '4/2564 - 4/2564', 'order_sell': '8 - 1', 'case_id': 'ผบ.508/2555', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '0', 'size2': '0', 'size3': '17', 'price': '931,000.00', 'tumbon': 'บางรักใหญ่(บางไผ่)', 'aumper': 'บางบัวทอง', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.508&law_suit_year=2555&Law_Court_ID=101&deed_no=41299&addrno=189/219', 'lat': '13.8914598689291', 'long': '100.4361107209', 'xprice_gov': '22,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '17.0'}, {'Law_Court_ID': '103', 'deed_no': '228106', 'addrno': '99/123', 'year': '4/2564 - 4/2564', 'order_sell': '9 - 1', 'case_id': 'ผบ.14995/2560', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '0', 'size2': '0', 'size3': '31.2', 'price': '1,886,280.00', 'tumbon': 'ไทรม้า', 'aumper': 'เมืองนนทบุรี', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.14995&law_suit_year=2560&Law_Court_ID=103&deed_no=228106&addrno=99/123', 'lat': '13.8617718192007', 'long': '100.4666147135', 'xprice_gov': '28,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '31.2'}, {'Law_Court_ID': '121', 'deed_no': '415104151141514', 'addrno': '132/42', 'year': '4/2564 - 4/2564', 'order_sell': '12 - 1', 'case_id': 'ผบ.16136/2559', 'prop_type': 'ห้องชุด', 'size1': '0', 'size2': '0', 'size3': '31.91', 'price': '303,145.00', 'tumbon': 'บางพูด', 'aumper': 'ปากเกร็ด', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.16136&law_suit_year=2559&Law_Court_ID=121&deed_no=41510%20%2041511%20%2041514&addrno=132/42', 'lat': '', 'long': '', 'xprice_gov': '', 'xarea1': '', 'xarea2': '', 'xarea3': ''}, {'Law_Court_ID': '101', 'deed_no': '68557', 'addrno': '99/224', 'year': '4/2564 - 4/2564', 'order_sell': '15 - 1', 'case_id': 'ผบ.4867/2562', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '0', 'size2': '0', 'size3': '18.7', 'price': '1,023,500.00', 'tumbon': 'บางพูด', 'aumper': 'ปากเกร็ด', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.4867&law_suit_year=2562&Law_Court_ID=101&deed_no=68557&addrno=99/224', 'lat': '13.9361549675151', 'long': '100.5123114300', 'xprice_gov': 'กรุณาตรวจสอบข้อมูลผ่านทาง', 'xarea1': '0', 'xarea2': '0', 'xarea3': '18.7'}, {'Law_Court_ID': '101', 'deed_no': '230868', 'addrno': '3/216', 'year': '4/2564 - 4/2564', 'order_sell': '16 - 1', 'case_id': 'ผบ.3206/2559', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '0', 'size2': '0', 'size3': '59.5', 'price': '4,244,500.00', 'tumbon': 'ท่าทราย', 'aumper': 'เมืองนนทบุรี', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.3206&law_suit_year=2559&Law_Court_ID=101&deed_no=230868&addrno=3/216', 'lat': '13.8706559317939', 'long': '100.5413897284', 'xprice_gov': '55,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '59.5'}, {'Law_Court_ID': '101', 'deed_no': '8801,139483', 'addrno': '115/1114', 'year': '4/2564 - 4/2564', 'order_sell': '18 - 1', 'case_id': 'ผบ.4352/2562', 'prop_type': 'ห้องชุด', 'size1': '0', 'size2': '0', 'size3': '29.16', 'price': '1,195,560.00', 'tumbon': 'ไทรม้า', 'aumper': 'เมืองนนทบุรี', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.4352&law_suit_year=2562&Law_Court_ID=101&deed_no=8801,%20139483&addrno=115/1114', 'lat': '13.8686911738563', 'long': '100.4683950619', 'xprice_gov': '8,800', 'xarea1': '10', 'xarea2': '2', 'xarea3': '60.0'}, {'Law_Court_ID': '101', 'deed_no': '66448', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 1', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '81', 'price': '1,620,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66448&addrno=-', 'lat': '13.9081902993764', 'long': '100.3692995762', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '81.0'}, {'Law_Court_ID': '101', 'deed_no': '66449', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 2', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '64', 'price': '1,280,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66449&addrno=-', 'lat': '13.9081899159931', 'long': '100.3694552705', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '64.0'}, {'Law_Court_ID': '101', 'deed_no': '66457', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 3', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '61', 'price': '1,220,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66457&addrno=-', 'lat': '13.9082786488564', 'long': '100.3708596703', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '61.0'}, {'Law_Court_ID': '101', 'deed_no': '66480', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '81', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66480&addrno=-', 'lat': '13.9076442286956', 'long': '100.3708909654', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '81.0'}, {'Law_Court_ID': '101', 'deed_no': '66497', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '54', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66497&addrno=-', 'lat': '13.907465327204', 'long': '100.3708996401', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '54.0'}, {'Law_Court_ID': '101', 'deed_no': '66498', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '88', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66498&addrno=-', 'lat': '13.9076376022997', 'long': '100.3710845409', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '88.0'}, {'Law_Court_ID': '101', 'deed_no': '66499', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '84', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66499&addrno=-', 'lat': '13.9076360628818', 'long': '100.3712577166', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '84.0'}, {'Law_Court_ID': '101', 'deed_no': '66503', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '57', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66503&addrno=-', 'lat': '13.9074587507417', 'long': '100.3714762822', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '57.0'}, {'Law_Court_ID': '101', 'deed_no': '66504', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '58', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66504&addrno=-', 'lat': '13.9073316239539', 'long': '100.3714769846', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '58.0'}, {'Law_Court_ID': '101', 'deed_no': '66518', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '56', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66518&addrno=-', 'lat': '13.9071826972287', 'long': '100.3691099667', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '56.0'}, {'Law_Court_ID': '101', 'deed_no': '66519', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '56', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66519&addrno=-', 'lat': '13.9070565053284', 'long': '100.3691093412', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '56.0'}, {'Law_Court_ID': '101', 'deed_no': '66520', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '88', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66520&addrno=-', 'lat': '13.9070939840441', 'long': '100.3693590450', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '88.0'}, {'Law_Court_ID': '101', 'deed_no': '66521', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '56', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66521&addrno=-', 'lat': '13.9071073844487', 'long': '100.3695260912', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '56.0'}, {'Law_Court_ID': '101', 'deed_no': '66522', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '56', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66522&addrno=-', 'lat': '13.9071159064738', 'long': '100.3696554683', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '56.0'}, {'Law_Court_ID': '101', 'deed_no': '66523', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '56', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66523&addrno=-', 'lat': '13.9071237287907', 'long': '100.3697848416', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '56.0'}, {'Law_Court_ID': '101', 'deed_no': '66524', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '72', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66524&addrno=-', 'lat': '13.907135960007', 'long': '100.3699293702', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '72.0'}, {'Law_Court_ID': '101', 'deed_no': '66528', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '72', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66528&addrno=-', 'lat': '13.9070731589731', 'long': '100.3708158376', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '72.0'}, {'Law_Court_ID': '101', 'deed_no': '66529', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '72', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66529&addrno=-', 'lat': '13.9070773041637', 'long': '100.3709626905', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '72.0'}, {'Law_Court_ID': '101', 'deed_no': '66530', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '72', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66530&addrno=-', 'lat': '13.9070832899', 'long': '100.3711106082', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '72.0'}, {'Law_Court_ID': '101', 'deed_no': '66531', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '72', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66531&addrno=-', 'lat': '13.9070906017104', 'long': '100.3712586299', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '72.0'}, {'Law_Court_ID': '101', 'deed_no': '66532', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '59', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66532&addrno=-', 'lat': '13.9072031543392', 'long': '100.3714779193', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '59.0'}, {'Law_Court_ID': '101', 'deed_no': '66533', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '59', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66533&addrno=-', 'lat': '13.9070742993842', 'long': '100.3714798128', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '59.0'}, {'Law_Court_ID': '101', 'deed_no': '66534', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '59', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66534&addrno=-', 'lat': '13.9069445017075', 'long': '100.3714815275', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '59.0'}, {'Law_Court_ID': '101', 'deed_no': '66535', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '59', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66535&addrno=-', 'lat': '13.9068136249241', 'long': '100.3714835062', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '59.0'}, {'Law_Court_ID': '101', 'deed_no': '66536', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '59', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66536&addrno=-', 'lat': '13.9066675899895', 'long': '100.3714854495', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '59.0'}, {'Law_Court_ID': '101', 'deed_no': '66537', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '90', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66537&addrno=-', 'lat': '13.9066851957079', 'long': '100.3710831049', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '90.0'}, {'Law_Court_ID': '101', 'deed_no': '66538', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '72', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66538&addrno=-', 'lat': '13.9067438672099', 'long': '100.3712520639', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '72.0'}, {'Law_Court_ID': '101', 'deed_no': '66539', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '74', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66539&addrno=-', 'lat': '13.9068767598636', 'long': '100.3712466780', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '74.0'}, {'Law_Court_ID': '101', 'deed_no': '66540', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '85', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66540&addrno=-', 'lat': '13.9068554876542', 'long': '100.3710698395', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '85.0'}, {'Law_Court_ID': '101', 'deed_no': '66541', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '79', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66541&addrno=-', 'lat': '13.9068474608237', 'long': '100.3709002851', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '79.0'}, {'Law_Court_ID': '101', 'deed_no': '66543', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '80', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66543&addrno=-', 'lat': '13.9068360057515', 'long': '100.3705705329', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '80.0'}, {'Law_Court_ID': '101', 'deed_no': '66545', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '71', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66545&addrno=-', 'lat': '13.9069190445253', 'long': '100.3700574101', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '71.0'}, {'Law_Court_ID': '101', 'deed_no': '66547', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '56', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66547&addrno=-', 'lat': '13.9069036524115', 'long': '100.3697775161', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '56.0'}, {'Law_Court_ID': '101', 'deed_no': '66548', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '56', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66548&addrno=-', 'lat': '13.9068957720083', 'long': '100.3696481816', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '56.0'}, {'Law_Court_ID': '101', 'deed_no': '66549', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '76', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66549&addrno=-', 'lat': '13.9068849018195', 'long': '100.3694953789', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '76.0'}, {'Law_Court_ID': '101', 'deed_no': '66550', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '65', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66550&addrno=-', 'lat': '13.9068735906106', 'long': '100.3693329555', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '65.0'}, {'Law_Court_ID': '101', 'deed_no': '66551', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '82', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66551&addrno=-', 'lat': '13.9069067247539', 'long': '100.3691064911', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '82.0'}, {'Law_Court_ID': '101', 'deed_no': '66552', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '68', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66552&addrno=-', 'lat': '13.9067182578627', 'long': '100.3690996955', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '68.0'}, {'Law_Court_ID': '101', 'deed_no': '66553', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '70', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66553&addrno=-', 'lat': '13.9067288767512', 'long': '100.3693355784', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '70.0'}, {'Law_Court_ID': '101', 'deed_no': '66554', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '77', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66554&addrno=-', 'lat': '13.9067414300724', 'long': '100.3695040638', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '77.0'}, {'Law_Court_ID': '101', 'deed_no': '66555', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '56', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66555&addrno=-', 'lat': '13.9067533622345', 'long': '100.3696563294', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '56.0'}, {'Law_Court_ID': '101', 'deed_no': '66556', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '56', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66556&addrno=-', 'lat': '13.9067626223268', 'long': '100.3697856479', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '56.0'}, {'Law_Court_ID': '101', 'deed_no': '66557', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '56', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66557&addrno=-', 'lat': '13.9067719639781', 'long': '100.3699148699', 'xprice_gov': '25,000', 'xarea1': '0', 'xarea2': '0', 'xarea3': '56.0'}]


def write_csv(file_name,page_data,csv_columns):
#     csv_file = file_name
    try:
        with open(file_name, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in page_data:
                writer.writerow(data)
    except IOError:
        print("I/O error")
        

file_name = "Names2.csv"
write_csv(file_name,page_data,csv_columns)

In [5]:
def find_gps(province,aumpur,choose_one_deed_no):
#     province = 'นนทบุรี'
#     aumpur = '02 - บางกรวย'
#     deed_no = '9805'

    # driver_path = "/Users/amit/Downloads/chromedriver"
 
#     a = ['01 - เมืองนนทบุรี','02 - บางกรวย','03 - บางใหญ่','04 - บางบัวทอง','05 - ไทรน้อย','06 - ปากเกร็ด','บ1 - บางปลา','ป1 - ตลาดขวัญ','พ1 - ตลิ่งชัน','พ7 - ตลาดขวัญ','พ8 - บางซื่อ']
    
#     for i in a:
#         if aumpur == i.split('-')[-1].strip():
#             aumpur = i

    
    try:
        url = 'http://dolwms.dol.go.th/tvwebp/'

        driver.get(url)


        driver.find_element_by_xpath('//*[@id="form1"]/div[6]/div/a[2]').click()
        # time.sleep(1)
        driver.find_element_by_xpath('//*[@id="ddlProvince"]').send_keys(province)
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="ddlAmphur"]').send_keys(aumpur)
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="txtPacelNo"]').send_keys(choose_one_deed_no)
        # time.sleep(2)
        driver.find_element_by_xpath('//*[@id="btnFind"]').click()

        time.sleep(1)
        # driver.page_source
        # driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[1]').text
        d = driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div').text

        d = d.split('\n')
    #     print(d)

    #     map_data = {
    #         'lat' : d[-6].split(',')[0].split(':')[1].strip(),
    #         'long' : d[-6].split(',')[1],
    #         'price_gov' : d[-8].split()[2],
    #         'area1' : d[-12].split()[2],
    #         'area2' : d[-12].split()[4],
    #         'area3' : d[-12].split()[6],
    #     }

        lat = d[-6].split(',')[0].split(':')[1].strip()
        long = d[-6].split(',')[1]
        xprice_gov = d[-8].split()[2]
        xarea1 = d[-12].split()[2]
        xarea2 = d[-12].split()[4]
        xarea3 = d[-12].split()[6]

        return lat,long,xprice_gov,xarea1,xarea2,xarea3
    except:
        print('Not find GPS...',province,aumpur,choose_one_deed_no)
        
# find_gps('นนทบุรี','04 - บางบัวทอง','160385')
find_gps('นนทบุรี', '06 - ปากเกร็ด', '68557')


Not find GPS... นนทบุรี 06 - ปากเกร็ด 68557


In [13]:
#ค้นหาแปลงที่ดิน
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions() 
options.add_argument("headless")
driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)


def find_gps(province,aumpur,deed_no):
#     province = 'นนทบุรี'
#     aumpur = '02 - บางกรวย'
#     deed_no = '9805'

    # driver_path = "/Users/amit/Downloads/chromedriver"
    url = 'http://dolwms.dol.go.th/tvwebp/'

    driver.get(url)


    driver.find_element_by_xpath('//*[@id="form1"]/div[6]/div/a[2]').click()
    # time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ddlProvince"]').send_keys(province)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ddlAmphur"]').send_keys(aumpur)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="txtPacelNo"]').send_keys(deed_no)
    # time.sleep(2)
    driver.find_element_by_xpath('//*[@id="btnFind"]').click()

    time.sleep(1)
    # driver.page_source
    # driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[1]').text
    d = driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div').text

    d = d.split('\n')
#     print(d)

    map_data = {
        'lat' : d[-6].split(',')[0].split(':')[1].strip(),
        'long' : d[-6].split(',')[1],
        'price_gov' : d[-8].split()[2],
        'area1' : d[-12].split()[2],
        'area2' : d[-12].split()[4],
        'area3' : d[-12].split()[6],
    }

    return map_data


province = 'นนทบุรี'
aumpur = '02 - บางกรวย'
deed_no = '9805'
    
print(find_gps(province,aumpur,deed_no))
    

# driver.get_attribute("innerHTML")


# //*[@id="ddlProvince"] นนทบุรี    12
# //*[@id="ddlAmphur"] 02 - บางกรวย      02
# //*[@id="txtPacelNo"]   10554
# //*[@id="btnFind"]   
# //*[@id="btnFind"]
# //*[@id="btnFind"]

# //*[@id="form1"]/div[6]/div/a[2]

# //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div

# //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[3]

<ipython-input-13-fde47fb7f3de>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)


UnexpectedAlertPresentException: Alert Text: None
Message: unexpected alert open: {Alert text : ไม่พบข้อมูลแปลงที่ดินที่ค้นหา!}
  (Session info: headless chrome=85.0.4183.83)
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 4.15.0-139-generic x86_64)


In [ ]:
##กรมบังคับคดี
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import csv

options = webdriver.ChromeOptions() 
options.add_argument("headless")
driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)

csv_columns = ['Law_Court_ID','deed_no','addrno','year','order_sell','case_id','prop_type','size1','size2','size3','price','tumbon','aumper','link']
csv_file = "x.csv"
with open(csv_file, 'a') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    
    
def find_percen_province(province):
    url = 'http://asset.led.go.th/newbidreg/'
    driver.get(url)
 
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div/div[3]/div/div[1]/table/tbody/tr/td[2]/select').send_keys(province)

    prov_percen = driver.current_url.split('=')[-1]

    return prov_percen

def find_maxpage(prov_percen):
    
    page = 1
    url = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)

    driver.get(url)
    max_page = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[2]/table[1]/tbody/tr/td[2]/div').text.split('/')[-1]
    print('max_page : ',max_page)
    return int(max_page)

def scraping(prov_percen,max_page):
#     dict_data = []
#     csv_file = "x.csv"
    for page in range(1,max_page):
        print('page : ',page)
        url = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)

        driver.get(url)

        html = driver.page_source
        # print(html)

        sup = BeautifulSoup(html, "html.parser").find("table",{"class":"table linkevent"}).find_all('tr')[2:]
        # print(sup)

        for i in sup:
            link = 'http://asset.led.go.th/newbidreg/'
            link += i['onclick'][13:-130].replace("ผบ", "%BC%BA").replace('บางส่วน','%BA%D2%A7%CA%E8%C7%B9').replace(' ','%20').replace('ไม่ปรากฏเลขทะเบียน','%E4%C1%E8%BB%C3%D2%A1%AF%E0%C5%A2%B7%D0%E0%BA%D5%C2%B9').replace('ม','C1')
            print(link)

            Law_Court_ID = link.split('&')[-3].split('=')[-1]
            deed_no = link.split('&')[-2].split('=')[-1].replace('%20','')
            addrno = link.split('&')[-1].split('=')[-1]


            ii = i.find_all('td')
            data = []
            for x in ii:
                data.append(x.text.strip())

            prop_data = {
                'Law_Court_ID' : Law_Court_ID,
                'deed_no' : deed_no,
                'addrno' : addrno,
                'year' : data[0],
                'order_sell' : data[1],
                'case_id' : data[2],
                'prop_type' : data[3],
                'size1' : data[4],
                'size2' : data[5],
                'size3' : data[6],
                'price' : data[7],
                'tumbon' : data[8],
                'aumper' : data[9],
                'link' : link
            }
#             'Law_Court_ID','deed_no','addrno','year','order_sell','case_id','prop_type','size1','size2','size3','price','tumbon','aumper','link'
            print(prop_data)
            print('------------------')
            try:
                with open(csv_file, 'a') as csvfile:
                    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
#                     writer.writeheader()
#                     for d in prop_data:
                    writer.writerow(prop_data)
            except IOError:
                print("I/O error")    
    
prov_percen = find_percen_province('นนทบุรี')
max_page = find_maxpage(prov_percen)
scraping(prov_percen,max_page)
    
    
    
    
    
    
    

    
    



            
#             dict_data.append(prop_data) 
    
#             try:
#                 with open(csv_file, 'a') as csvfile:
#                     writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
#                     writer.writeheader()
#                     for d in prop_data:
#                         writer.writerow(d)
#             except IOError:
#                 print("I/O error")   
# d = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[2]/div[2]/table/tbody').text.split('\n')


# links = driver.find_elements_by_partial_link_text('##')
# for link in links:
#     print('lol : ',link.get_attribute("href"))


#     print('Law_Court_ID : ',Law_Court_ID)
#     print('deed_no : ',deed_no)
#     print('addrno : ',addrno)    
    
  # driver.find_element_by_xpath('//*[@id="form1"]/div[6]/div/a[2]').click()
    # time.sleep(1)
    # driver.find_element_by_xpath('//*[@id="ddlProvince"]').send_keys('นนทบุรี')
    # driver.find_element_by_xpath('//*[@id="prov"]').click()
    # time.sleep(1)

# driver.find_element_by_css_selector('#tbody')



# for i in d:
#     print(i.split())
#     data = i.split()
#     prop_data = {
#         'Law_Court_ID' : data[6],
#         'deed_no' : data[0],
#         'addrno' : data[0],
#         'year' : data[0],
#         'order_sell' : data[1],
#         'case_id' : data[2],
#         'prop_type' : data[3],
#         'size1' : data[4],
#         'size2' : data[5],
#         'size3' : data[6],
#         'price' : data[7],
#         'tumbon' : data[8],
#         'aumper' : data[9],
# #         'link' : link
#     }
#     print(prop_data)






# page = 1
# try :
# # url_naweb = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok'
# url_naweb = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)
# print(url_naweb)

# r = requests.get(url_naweb)
# r.encoding = 'utf-8' # กำหนด encoding กันปัญหาการถอดรหัสผิดแบบ


# sup = BeautifulSoup(r.content, 'html.parser').find("table",{"class":"table linkevent"}).find_all('tr')[2:]

# for i in sup:
#     link = 'http://asset.led.go.th/newbidreg/'
#     link += i['onclick'][13:-130].replace("ผบ", "%BC%BA").replace(' ','%20')
#     print(link)
#     Law_Court_ID = link.split('&')[-3].split('=')[-1]
#     deed_no = link.split('&')[-2].split('=')[-1]
#     addrno = link.split('&')[-1].split('=')[-1]
    
#     ii = i.find_all('td')
#     data = []
#     for x in ii:
#         data.append(x.text.strip())
#     prop_data = {
#         'Law_Court_ID' : Law_Court_ID,
#         'deed_no' : deed_no,
#         'addrno' : addrno,
#         'year' : data[0],
#         'order_sell' : data[1],
#         'case_id' : data[2],
#         'prop_type' : data[3],
#         'size1' : data[4],
#         'size2' : data[5],
#         'size3' : data[6],
#         'price' : data[7],
#         'tumbon' : data[8],
#         'aumper' : data[9],
#         'link' : link
#     }
#     print(prop_data)
#     print('------------------')












<ipython-input-3-648e233cb593>:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)


In [ ]:
# http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66422&addrno=%E4%C1%E8%BB%C3%D2%A1%AF%E0%C5%A2%B7%D0%E0%BA%D5%C2%B9
# http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66422&addrno=ไม่ปรากฏเลขทะเบียน  
    
# http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.4280&law_suit_year=2561&Law_Court_ID=121&deed_no=149458&addrno=78/475%20ม.9
# http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.4280&law_suit_year=2561&Law_Court_ID=121&deed_no=149458&addrno=78/475%20%C1.9   
    
# http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.18670&law_suit_year=2560&Law_Court_ID=121&deed_no=178896,178897%20(%BA%D2%A7%CA%E8%C7%B9)&addrno=9/188
# http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.18670&law_suit_year=2560&Law_Court_ID=121&deed_no=178896,178897%20(บางส่วน)&addrno=9/188    

In [ ]:
##กรมบังคับคดี
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
url_naweb = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=%B9%B9%B7%BA%D8%C3%D5&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok'

r = requests.get(url_naweb)
r.encoding = 'utf-8' # กำหนด encoding กันปัญหาการถอดรหัสผิดแบบ

sup = BeautifulSoup(r.content, 'html.parser').find("table",{"class":"table linkevent"}).find_all('tr')[2:]

for i in sup:
    link = 'http://asset.led.go.th/newbidreg/'
    link += i['onclick'][13:-130].replace("ผบ", "%BC%BA").replace(' ','%20')
#     print(link)
    Law_Court_ID = link.split('&')[-3].split('=')[-1]
    deed_no = link.split('&')[-2].split('=')[-1]
    addrno = link.split('&')[-1].split('=')[-1]
#     print('Law_Court_ID : ',Law_Court_ID)
#     print('deed_no : ',deed_no)
#     print('addrno : ',addrno)
    
    ii = i.find_all('td')
    data = []
    for x in ii:
        data.append(x.text.strip())
#     print(data)
    prop_data = {
        'Law_Court_ID' : Law_Court_ID,
        'deed_no' : deed_no,
        'addrno' : addrno,
        'year' : data[0],
        'order_sell' : data[1],
        'case_id' : data[2],
        'prop_type' : data[3],
        'size1' : data[4],
        'size2' : data[5],
        'size3' : data[6],
        'price' : data[7],
        'tumbon' : data[8],
        'aumper' : data[9],
        'link' : link
    }
    print(prop_data)
    print('------------------')

In [2]:
#ค้นหาแปลงที่ดิน
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions() 
options.add_argument("headless")
driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)


def find_gps(province,aumpur,deed_no):
#     province = 'นนทบุรี'
#     aumpur = '02 - บางกรวย'
#     deed_no = '9805'

    # driver_path = "/Users/amit/Downloads/chromedriver"
    url = 'http://dolwms.dol.go.th/tvwebp/'

    driver.get(url)


    driver.find_element_by_xpath('//*[@id="form1"]/div[6]/div/a[2]').click()
    # time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ddlProvince"]').send_keys(province)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ddlAmphur"]').send_keys(aumpur)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="txtPacelNo"]').send_keys(deed_no)
    # time.sleep(2)
    driver.find_element_by_xpath('//*[@id="btnFind"]').click()

    time.sleep(1)
    # driver.page_source
    # driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[1]').text
    d = driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div').text

    d = d.split('\n')
#     print(d)

    map_data = {
        'lat' : d[-6].split(',')[0].split(':')[1].strip(),
        'long' : d[-6].split(',')[1],
        'price_gov' : d[-8].split()[2],
        'area1' : d[-12].split()[2],
        'area2' : d[-12].split()[4],
        'area3' : d[-12].split()[6],
    }

    return map_data


province = 'นนทบุรี'
aumpur = '02 - บางกรวย'
deed_no = '9805'
    
print(find_gps(province,aumpur,deed_no))
    

# driver.get_attribute("innerHTML")


# //*[@id="ddlProvince"] นนทบุรี    12
# //*[@id="ddlAmphur"] 02 - บางกรวย      02
# //*[@id="txtPacelNo"]   10554
# //*[@id="btnFind"]   
# //*[@id="btnFind"]
# //*[@id="btnFind"]

# //*[@id="form1"]/div[6]/div/a[2]

# //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div

# //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[3]

<ipython-input-2-fde47fb7f3de>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)


{'lat': '13.8287281871677', 'long': '100.3945950334', 'price_gov': '18,000', 'area1': '0', 'area2': '0', 'area3': '76.5'}


In [ ]:
##link ok
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
url_naweb = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=%B9%B9%B7%BA%D8%C3%D5&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok'

r = requests.get(url_naweb)
r.encoding = 'utf-8' # กำหนด encoding กันปัญหาการถอดรหัสผิดแบบ
sup = BeautifulSoup(r.text,'lxml').find("table",{"class":"table linkevent"}).find_all('tr')[2:]

for i in sup:
#     url_link = 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.' 
#     url_link += i['onclick'][12:-16].split(',')[0].split('.')[2].split("'")[0]
#     print(url_link)
    a = i['onclick'][43:-130].replace(" ", "%20")
#     print(a)
    print('http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.'+a)

In [ ]:
##link ok
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
url_naweb = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=%B9%B9%B7%BA%D8%C3%D5&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok'

r = requests.get(url_naweb)
r.encoding = 'utf-8' # กำหนด encoding กันปัญหาการถอดรหัสผิดแบบ

sup = BeautifulSoup(r.content, 'html.parser').find("table",{"class":"table linkevent"}).find_all('tr')[2:]

for i in sup:
    link = 'http://asset.led.go.th/newbidreg/'
    link += i['onclick'][13:-130].replace("ผบ", "%BC%BA").replace(' ','%20')
#     print(link)
    Law_Court_ID = link.split('&')[-3].split('=')[-1]
    deed_no = link.split('&')[-2].split('=')[-1]
    addrno = link.split('&')[-1].split('=')[-1]
#     print('Law_Court_ID : ',Law_Court_ID)
#     print('deed_no : ',deed_no)
#     print('addrno : ',addrno)
    
    ii = i.find_all('td')
    data = []
    for x in ii:
        data.append(x.text.strip())
#     print(data)
    prop_data = {
        'Law_Court_ID' : Law_Court_ID,
        'deed_no' : deed_no,
        'addrno' : addrno,
        'year' : data[0],
        'order_sell' : data[1],
        'case_id' : data[2],
        'prop_type' : data[3],
        'size1' : data[4],
        'size2' : data[5],
        'size3' : data[6],
        'price' : data[7],
        'tumbon' : data[8],
        'aumper' : data[9],
        'link' : link
    }
    print(prop_data)
    print(link)
    print('------------------')


In [ ]:
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.508&law_suit_year=2555&Law_Court_ID=101&deed_no=41299&addrno=189/219


len("','_blank',null,'width=1300,height=900,status=yes,toolbar=no,menubar=yes,location=no,scrollbars=yes,resizable=yes'); return false;")

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
# url = requests.get("https://pantip.com/topic/37319282")
url = requests.get("http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=%B9%B9%B7%BA%D8%C3%D5&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok")
soup = BeautifulSoup(url.content, "html.parser")

In [ ]:
data

In [ ]:
import requests
from bs4 import BeautifulSoup

# r = requests.get("https://coins.co.th/")
# soup = BeautifulSoup(r.content, "html.parser")

url = requests.get("http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=%B9%B9%B7%BA%D8%C3%D5&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok")
soup = BeautifulSoup(url.content, "html.parser")

data = soup.find_all("tbody")
print(data)

In [ ]:
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
url_naweb = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=%B9%B9%B7%BA%D8%C3%D5&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok'

r = requests.get(url_naweb)
r.encoding = 'utf-8' # กำหนด encoding กันปัญหาการถอดรหัสผิดแบบ
sup = BeautifulSoup(r.text,'lxml').find("table",{"class":"table linkevent"}).find_all('tr')[2:]

for i in sup:
#     url_link = 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.' 
#     url_link += i['onclick'][12:-16].split(',')[0].split('.')[2].split("'")[0]
#     print(url_link)
    a = i['onclick'][43:-130].replace(" ", "%20")
#     print(a)
    print('http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.'+a)
#     a = quote('deed_no=41510  41511  41514',safe='=')

    print('--------------------------------')

# print(type(sup))

# print(sup.find_all('table'))
# print(sup)


# table class="table linkevent"

# data = soup.find("h2",{"class":"display-post-title"})
# deed_no=41510%20%2041511%20%2041514&addrno=132/42

In [ ]:
# soup = BeautifulSoup(html_doc, 'html.parser')
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.14800&law_suit_year=2561&Law_Court_ID=121&deed_no=160385,160386,160387,160388,160389,160390,160393,160394,160395,160396,160397,160&addrno=351/56

In [ ]:
from urllib.parse import quote
a = quote('deed_no=41510  41511  41514',safe='=')
a
# deed_no=41510%20%2041511%20%2041514&addrno=132/42

In [ ]:
len(",'_blank',null,'width=1300,height=900,status=yes,toolbar=no,menubar=yes,location=no,scrollbars=yes,resizable=yes'); return false;")

In [ ]:
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66522&addrno=-

In [ ]:
# https://stackoverflow.com/questions/53973423/getting-value-after-button-click-with-beautifulsoup-python

from selenium.webdriver.support.ui import WebDriverWait as wait

def open_browser():
    driver = webdriver.Chrome("/home/felipe/Downloads/chromedriver")
    driver.get(url)
    driver.find_element_by_id('bt_gerar_cpf').click()
    text_field = driver.find_element_by_id('texto_cpf')
    text = wait(driver, 10).until(lambda driver: not text_field.text == 'Gerando...' and text_field.text)
    return text

print(open_browser())

In [ ]:
import requests

url = 'https://www.4devs.com.br/ferramentas_online.php'
data = {'acao': 'gerar_cpf', 'pontuacao': 'S'}
response = requests.post(url, data=data)
print(response.text)

In [ ]:
import requests

url = 'http://dolwms.dol.go.th/tvwebp/'
data = {'ddlProvince':'12','ddlAmphur':'06','txtPacelNo': '41510'}
# นนทบุรี

response = requests.post(url, data=data)
print(response.text)

In [ ]:
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions() 
options.add_argument("headless")

driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)

# driver_path = "/Users/amit/Downloads/chromedriver"
url = 'http://dolwms.dol.go.th/tvwebp/'

driver.get(url)


driver.find_element_by_xpath('//*[@id="form1"]/div[6]/div/a[2]').click()
# time.sleep(1)
driver.find_element_by_xpath('//*[@id="ddlProvince"]').send_keys('นนทบุรี')
time.sleep(1)
driver.find_element_by_xpath('//*[@id="ddlAmphur"]').send_keys('02 - บางกรวย')
time.sleep(1)
driver.find_element_by_xpath('//*[@id="txtPacelNo"]').send_keys('9805')
# time.sleep(2)
driver.find_element_by_xpath('//*[@id="btnFind"]').click()

time.sleep(1)
# driver.page_source
# driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[1]').text
d = driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div').text

d = d.split('\n')
print(d)

map_data = {
    'lat' : d[-6].split(',')[0].split(':')[1].strip(),
    'long' : d[-6].split(',')[1],
    'price_gov' : d[-8].split()[2],
    'area1' : d[-12].split()[2],
    'area2' : d[-12].split()[4],
    'area3' : d[-12].split()[6],
}

map_data

# driver.get_attribute("innerHTML")


# //*[@id="ddlProvince"] นนทบุรี    12
# //*[@id="ddlAmphur"] 02 - บางกรวย      02
# //*[@id="txtPacelNo"]   10554
# //*[@id="btnFind"]   
# //*[@id="btnFind"]
# //*[@id="btnFind"]

# //*[@id="form1"]/div[6]/div/a[2]

# //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div

# //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[3]

In [ ]:
# v1
# # import time

# # from selenium import webdriver
# # from selenium.webdriver.common.keys import Keys
# # from selenium.webdriver.common.by import By

# # driver = webdriver.Chrome('/usr/bin/chromedriver')

# # # driver_path = "/Users/amit/Downloads/chromedriver"
# # url = 'http://dolwms.dol.go.th/tvwebp/'

# # driver.get(url)


# # driver.find_element_by_xpath('//*[@id="form1"]/div[6]/div/a[2]').click()
# # # time.sleep(1)
# # driver.find_element_by_xpath('//*[@id="ddlProvince"]').send_keys('นนทบุรี')


# # time.sleep(1)
# # driver.find_element_by_xpath('//*[@id="ddlAmphur"]').send_keys('02 - บางกรวย')
# # time.sleep(1)
# # driver.find_element_by_xpath('//*[@id="txtPacelNo"]').send_keys('9805')
# # # time.sleep(2)
# # driver.find_element_by_xpath('//*[@id="btnFind"]').click()

# # time.sleep(1)
# # # driver.page_source
# # # driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[1]').text
# # driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div').text

# # # driver.get_attribute("innerHTML")


# # # //*[@id="ddlProvince"] นนทบุรี    12
# # # //*[@id="ddlAmphur"] 02 - บางกรวย      02
# # # //*[@id="txtPacelNo"]   10554
# # # //*[@id="btnFind"]   
# # # //*[@id="btnFind"]
# # # //*[@id="btnFind"]

# # # //*[@id="form1"]/div[6]/div/a[2]

# # # //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div

# # # //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[3]